In [37]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import StateGraph, END, START, add_messages
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, SystemMessage, BaseMessage
from typing import TypedDict, Annotated

In [38]:
load_dotenv()
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

In [47]:
class ChatbotSchema(TypedDict):
    message: Annotated[list[BaseMessage], add_messages]

In [48]:
def func(state: ChatbotSchema):

    message = state["message"]
    response = llm.invoke(message)
    return {"message": [response]}

In [49]:
graph = StateGraph(ChatbotSchema)

graph.add_node("func",func)

graph.add_edge(START,"func")
graph.add_edge("func",END)

workflow = graph.compile()

In [53]:
workflow.invoke({"message":"hii"})["message"][-1].content

'Hi there! How can I help you today?'